In [22]:
import pandas as pd

df_crime_yearly = pd.read_csv('../final_output/final_output_csv/ouput_2crime_yearly_2014_2024.csv', encoding='cp949')
df_time_final = pd.read_csv('../final_output/final_output_csv/ouput_2crime_time_distribution.csv', encoding='cp949')
df_place_final = pd.read_csv('../final_output/final_output_csv/ouput_2crime_place_distribution.csv', encoding='cp949')

print("준비완료")

준비완료


In [21]:
"""
서울시 5대 범죄 위험 시간대 가설 검증 및 시각화
(한글 폰트 문제 해결 버전)
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ============================================
# 한글 폰트 자동 설정 (macOS/Linux/Windows 호환)
# ============================================

def set_korean_font():
    """시스템에서 사용 가능한 한글 폰트 자동 감지 및 설정"""
    
    # 폰트 우선순위 리스트 (macOS → Linux → Windows)
    font_candidates = [
        'AppleGothic',           # macOS 기본
        'Apple SD Gothic Neo',   # macOS
        'NanumGothic',          # 나눔고딕
        'NanumBarunGothic',     # 나눔바른고딕
        'Malgun Gothic',        # 맑은 고딕 (Windows)
        'Noto Sans CJK KR',     # 구글 Noto
        'DejaVu Sans'           # 폴백 (한글 안되지만 깨지지는 않음)
    ]
    
    # 시스템에 설치된 폰트 목록
    available_fonts = [f.name for f in fm.fontManager.ttflist]
    
    # 사용 가능한 첫 번째 폰트 찾기
    selected_font = None
    for font in font_candidates:
        if font in available_fonts:
            selected_font = font
            break
    
    # 폰트 설정
    if selected_font:
        plt.rc('font', family=selected_font)
        print(f"✅ 한글 폰트 설정: {selected_font}")
    else:
        # 폰트를 찾지 못한 경우 - 유니코드 사용
        plt.rcParams['axes.unicode_minus'] = False
        print("⚠️ 한글 폰트를 찾지 못했습니다. 기본 설정을 사용합니다.")
        print("   한글이 깨질 수 있습니다. 나눔고딕 설치를 권장합니다.")
    
    # 마이너스 기호 깨짐 방지
    plt.rcParams['axes.unicode_minus'] = False
    
    return selected_font

# 1. Seaborn 스타일을 '가장 먼저' 설정 
# (스타일 설정이 폰트 세팅을 초기화하는 것을 방지하기 위함이야)
sns.set_style("whitegrid")

# 2. 그 다음 한글 폰트 설정 함수 호출
selected_font = set_korean_font()

# 3. 폰트 설정 후 rcParams에 명시적으로 한 번 더 박아주기
if selected_font:
    plt.rcParams['font.family'] = selected_font
    plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지
    plt.rcParams['figure.figsize'] = (16, 10)   # 그래프 기본 크기 설정
    print(f"✅ 최종 폰트 적용 완료: {selected_font}")
plt.rcParams['figure.figsize'] = (16, 10)


print("="*80)
print("📊 서울시 5대 범죄 위험 시간대 가설 검증")
print("="*80)


✅ 한글 폰트 설정: AppleGothic
✅ 최종 폰트 적용 완료: AppleGothic
📊 서울시 5대 범죄 위험 시간대 가설 검증


In [23]:
# ============================================
# 데이터 로드
# ============================================

df_time = pd.read_csv('../final_output/final_output_csv/ouput_2crime_time_distribution.csv', encoding='cp949')
df_place = pd.read_csv('../final_output/final_output_csv/ouput_2crime_place_distribution.csv', encoding='cp949')

print("\n✅ 데이터 로드 완료")
print(f"   - 시간 분포: {df_time.shape}")
print(f"   - 장소 분포: {df_place.shape}")


✅ 데이터 로드 완료
   - 시간 분포: (5, 10)
   - 장소 분포: (5, 5)


In [24]:
# ============================================
# 1. 장소별 범죄 유형 분포 분석
# ============================================

print("\n" + "="*80)
print("[분석 1] 장소별 범죄 유형 분포")
print("="*80)

# 데이터 준비
place_cols = ['주거지구', '업무지구', '다중이용시설', '기타']
df_place_analysis = df_place.set_index('범죄분류')[place_cols]

# 비율 계산
df_place_pct = df_place_analysis.div(df_place_analysis.sum(axis=0), axis=1) * 100

print("\n📊 장소별 범죄 비율 (%):")
print(df_place_pct.round(2))

# 1-1. Heatmap
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Heatmap - 절대값
sns.heatmap(df_place_analysis, annot=True, fmt=',', cmap='YlOrRd', 
            cbar_kws={'label': 'Count'}, ax=axes[0], linewidths=0.5)
axes[0].set_title('Place-wise Crime Count (Absolute)', fontsize=16, weight='bold', pad=20)
axes[0].set_xlabel('Location Category', fontsize=12, weight='bold')
axes[0].set_ylabel('Crime Type', fontsize=12, weight='bold')

# Heatmap - 비율
sns.heatmap(df_place_pct, annot=True, fmt='.1f', cmap='RdYlGn_r', 
            cbar_kws={'label': 'Percentage (%)'}, ax=axes[1], linewidths=0.5)
axes[1].set_title('Place-wise Crime Ratio (%)', fontsize=16, weight='bold', pad=20)
axes[1].set_xlabel('Location Category', fontsize=12, weight='bold')
axes[1].set_ylabel('Crime Type', fontsize=12, weight='bold')

plt.tight_layout()
plt.savefig('../final_output/01_place_crime_heatmap_fixed.png', dpi=300, bbox_inches='tight')
print("\n✅ 그래프 저장: 01_place_crime_heatmap_fixed.png")
plt.close()

# 1-2. Stacked Bar Chart
fig, ax = plt.subplots(figsize=(14, 8))

# 영문 라벨 사용 (한글 깨짐 방지)
df_place_pct_eng = df_place_pct.copy()
df_place_pct_eng.index = ['Theft', 'Murder', 'Robbery', 'Sex Crime', 'Violence']

df_place_pct_eng.T.plot(kind='bar', stacked=True, ax=ax, 
                         color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8'])

ax.set_title('Crime Type Distribution by Location', fontsize=18, weight='bold', pad=20)
ax.set_xlabel('Location Category', fontsize=14, weight='bold')
ax.set_ylabel('Percentage (%)', fontsize=14, weight='bold')
ax.legend(title='Crime Type', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=11)
ax.set_xticklabels(ax.get_xticklabels(), rotation=0, ha='center')
ax.grid(axis='y', alpha=0.3)

# 각 바에 비율 표시
for container in ax.containers:
    ax.bar_label(container, fmt='%.1f%%', label_type='center', fontsize=9, weight='bold')

plt.tight_layout()
plt.savefig('../final_output/02_place_crime_stacked_fixed.png', dpi=300, bbox_inches='tight')
print("✅ 그래프 저장: 02_place_crime_stacked_fixed.png")
plt.close()

# 주요 인사이트 추출
print("\n📌 [인사이트 1] 장소별 주요 범죄")
print("-"*80)
for place in place_cols:
    top_crime = df_place_pct[place].idxmax()
    top_pct = df_place_pct[place].max()
    print(f"   {place:12s}: {top_crime} (비율: {top_pct:.1f}%)")




[분석 1] 장소별 범죄 유형 분포

📊 장소별 범죄 비율 (%):
       주거지구   업무지구  다중이용시설     기타
범죄분류                             
절도    31.10  40.14   45.78  56.92
살인     0.48   0.13    0.10   0.25
강도     0.24   0.14    0.19   0.20
성폭력    8.34   8.70    6.82   9.44
폭력    59.84  50.88   47.10  33.18

✅ 그래프 저장: 01_place_crime_heatmap_fixed.png
✅ 그래프 저장: 02_place_crime_stacked_fixed.png

📌 [인사이트 1] 장소별 주요 범죄
--------------------------------------------------------------------------------
   주거지구        : 폭력 (비율: 59.8%)
   업무지구        : 폭력 (비율: 50.9%)
   다중이용시설      : 폭력 (비율: 47.1%)
   기타          : 절도 (비율: 56.9%)


In [25]:
# ============================================
# 2. 범죄 유형별 시간대별 발생 패턴
# ============================================

print("\n" + "="*80)
print("[분석 2] 범죄 유형별 시간대별 발생 패턴")
print("="*80)

# 시간대 컬럼 정리
time_cols = [col for col in df_time.columns if col != '범죄분류' and col != '미상']

# 데이터 준비
df_time_analysis = df_time.set_index('범죄분류')[time_cols]

# 시간대 라벨
time_labels = ['00-02', '03-05', '06-08', '09-11', '12-14', '15-17', '18-20', '21-23']

print("\n📊 범죄별 시간대별 발생 건수:")
print(df_time_analysis)

# 영문 범죄명 매핑
crime_name_map = {
    '절도': 'Theft',
    '살인': 'Murder',
    '강도': 'Robbery',
    '성폭력': 'Sex Crime',
    '폭력': 'Violence'
}

# 2-1. Line Chart - 절대값
fig, ax = plt.subplots(figsize=(16, 8))

for crime in df_time_analysis.index:
    crime_eng = crime_name_map.get(crime, crime)
    ax.plot(time_labels, df_time_analysis.loc[crime], 
            marker='o', linewidth=2.5, markersize=8, label=crime_eng)

ax.set_title('Crime Pattern by Time (Absolute Count)', fontsize=18, weight='bold', pad=20)
ax.set_xlabel('Time Period (Hour)', fontsize=14, weight='bold')
ax.set_ylabel('Crime Count', fontsize=14, weight='bold')
ax.legend(title='Crime Type', fontsize=12, title_fontsize=13, loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_ylim(bottom=0)

# 피크 타임 표시
for crime, crime_eng in [('절도', 'Theft'), ('폭력', 'Violence'), ('강도', 'Robbery'), ('성폭력', 'Sex Crime'), ('폭력', 'Violence')]:
    peak_idx = df_time_analysis.loc[crime].argmax()
    peak_value = df_time_analysis.loc[crime].max()
    ax.annotate(f'{crime_eng} Peak\n{peak_value:,}', 
                xy=(peak_idx, peak_value), 
                xytext=(peak_idx, peak_value + 5000),
                ha='center', fontsize=10, weight='bold',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7),
                arrowprops=dict(arrowstyle='->', lw=2, color='red'))

plt.tight_layout()
plt.savefig('../final_output/03_time_pattern_absolute_fixed.png', dpi=300, bbox_inches='tight')
print("\n✅ 그래프 저장: 03_time_pattern_absolute_fixed.png")
plt.close()

# 2-2. Line Chart - 정규화
df_time_pct = df_time_analysis.div(df_time_analysis.sum(axis=1), axis=0) * 100

fig, ax = plt.subplots(figsize=(16, 8))

for crime in df_time_pct.index:
    crime_eng = crime_name_map.get(crime, crime)
    ax.plot(time_labels, df_time_pct.loc[crime], 
            marker='o', linewidth=2.5, markersize=8, label=crime_eng)

ax.set_title('Crime Pattern by Time (Percentage)', fontsize=18, weight='bold', pad=20)
ax.set_xlabel('Time Period (Hour)', fontsize=14, weight='bold')
ax.set_ylabel('Percentage (%)', fontsize=14, weight='bold')
ax.legend(title='Crime Type', fontsize=12, title_fontsize=13, loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_ylim(bottom=0)

plt.tight_layout()
plt.savefig('../final_output/04_time_pattern_percentage_fixed.png', dpi=300, bbox_inches='tight')
print("✅ 그래프 저장: 04_time_pattern_percentage_fixed.png")
plt.close()

# 피크 타임 분석
print("\n📌 [인사이트 2] 범죄별 피크 타임")
print("-"*80)
for crime in df_time_analysis.index:
    peak_idx = df_time_analysis.loc[crime].argmax()
    peak_time = time_labels[peak_idx]
    peak_count = df_time_analysis.loc[crime].max()
    
    min_idx = df_time_analysis.loc[crime].argmin()
    min_time = time_labels[min_idx]
    min_count = df_time_analysis.loc[crime].min()
    
    print(f"   {crime:8s}: Peak {peak_time} ({peak_count:>6,}) / Low {min_time} ({min_count:>6,})")



[분석 2] 범죄 유형별 시간대별 발생 패턴

📊 범죄별 시간대별 발생 건수:
      00시00분-02시59분  03시00분-05시59분  06시00분-08시59분  09시00분-11시59분  \
범죄분류                                                               
절도            13095          14176          13104          21763   
살인               49             73             49             86   
강도              112            133             45             66   
성폭력            3061           3332           2233           2317   
폭력            24542          20946          12080          15184   

      12시00분-14시59분  15시00분-17시59분  18시00분-20시59분  21시00분-23시59분  
범죄분류                                                              
절도            27242          28582          27272          20587  
살인               97             96            110            132  
강도               75             72             86            112  
성폭력            2539           3204           3920           5048  
폭력            16114          19065          26120          42840  

✅ 그래프 저장

In [26]:
# ============================================
# 3. 가설 검증 통합 분석
# ============================================

print("\n\n" + "="*80)
print("[분석 3] 가설 검증 - 통합 분석")
print("="*80)

# 3-1. 장소별 주요 범죄 식별
print("\n🔍 STEP 1: 각 장소에서 가장 많이 발생하는 범죄 식별")
print("-"*80)

place_main_crimes = {}
for place in ['주거지구', '업무지구', '다중이용시설', '기타']:
    place_data = df_place.set_index('범죄분류')[place]
    top_crimes = place_data.nlargest(5)
    place_main_crimes[place] = top_crimes
    
    print(f"\n   {place}:")
    for crime, count in top_crimes.items():
        pct = (count / place_data.sum()) * 100
        print(f"      {crime}: {count:>6,} ({pct:>5.1f}%)")

# 3-2. 피크 타임 분석
print("\n\n🔍 STEP 2: 주요 범죄의 피크 타임 분석")
print("-"*80)

crime_peak_times = {}
for crime in df_time['범죄분류']:
    crime_data = df_time_analysis.loc[crime]
    peak_idx = crime_data.argmax()
    peak_time = time_labels[peak_idx]
    peak_count = crime_data.max()
    crime_peak_times[crime] = (peak_time, peak_count)
    
    print(f"   {crime:8s}: {peak_time} ({peak_count:,})")

# 3-3. 통합 시각화 (영문 버전)
fig = plt.figure(figsize=(20, 16))
gs = fig.add_gridspec(4, 2, hspace=0.3, wspace=0.3)


# # 서브플롯 1: 장소별 범죄 구성
# ax1 = fig.add_subplot(gs[0, :])
# df_place_pct_eng[['주거지구', '업무지구', '다중이용시설', '기타']].T.plot(
#     kind='bar', stacked=True, ax=ax1,
#     color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8'])
# ax1.set_title('Crime Composition by Location', fontsize=16, weight='bold')
# ax1.set_xlabel('Location Category', fontsize=12, weight='bold')
# ax1.set_ylabel('Percentage (%)', fontsize=12, weight='bold')
# ax1.set_xticklabels(['Residential', 'Business', 'Public Facility', 'Others'], rotation=0)
# ax1.legend(title='Crime Type', bbox_to_anchor=(1.05, 1), loc='upper left')
# ax1.grid(axis='y', alpha=0.3)

# 서브플롯 2-4: 장소별 시간 패턴
locations = [
    ('주거지구', 'Residential', gs[1, 0]),
    ('업무지구', 'Business', gs[1, 1]),
    ('다중이용시설', 'Public Facility', gs[2, 0]),
    ('기타', 'Others', gs[2, 1] )
]

for place_kr, place_en, position in locations:
    ax = fig.add_subplot(position)
    for crime in place_main_crimes[place_kr].index:
        crime_eng = crime_name_map.get(crime, crime)
        ax.plot(time_labels, df_time_analysis.loc[crime], 
                marker='o', linewidth=2.5, markersize=8, label=crime_eng)
    ax.set_title(f'{place_en} Crime Time Pattern', fontsize=14, weight='bold')
    ax.set_xlabel('Time Period', fontsize=11)
    ax.set_ylabel('Crime Count', fontsize=11)
    ax.legend()
    ax.grid(True, alpha=0.3)

    plt.savefig('../final_output/05_hypothesis_verification_fixed.png', dpi=300, bbox_inches='tight')
print("\n✅ 통합 그래프 저장: 05_hypothesis_verification_fixed.png")
plt.close()



[분석 3] 가설 검증 - 통합 분석

🔍 STEP 1: 각 장소에서 가장 많이 발생하는 범죄 식별
--------------------------------------------------------------------------------

   주거지구:
      폭력: 49,634 ( 59.8%)
      절도: 25,795 ( 31.1%)
      성폭력:  6,917 (  8.3%)
      살인:    402 (  0.5%)
      강도:    195 (  0.2%)

   업무지구:
      폭력:  7,733 ( 50.9%)
      절도:  6,101 ( 40.1%)
      성폭력:  1,323 (  8.7%)
      강도:     22 (  0.1%)
      살인:     20 (  0.1%)

   다중이용시설:
      폭력: 124,774 ( 47.1%)
      절도: 121,287 ( 45.8%)
      성폭력: 18,074 (  6.8%)
      강도:    507 (  0.2%)
      살인:    274 (  0.1%)

   기타:
      절도: 34,446 ( 56.9%)
      폭력: 20,080 ( 33.2%)
      성폭력:  5,715 (  9.4%)
      살인:    151 (  0.2%)
      강도:    121 (  0.2%)


🔍 STEP 2: 주요 범죄의 피크 타임 분석
--------------------------------------------------------------------------------
   절도      : 15-17 (28,582)
   살인      : 21-23 (132)
   강도      : 03-05 (133)
   성폭력     : 21-23 (5,048)
   폭력      : 21-23 (42,840)

✅ 통합 그래프 저장: 05_hypothesis_verification_fixed.png
